# Clasificador Bayesiano Inocente (Naive)

Algoritmo conceptual y se dice que es inocente porque se presupone que las características son indepndientes una de la otra. Se comporta bien cuando se tienen variables categóricas y es bastante bueno cuando se tienen variables categóricas con gran cantidad de categorías. No es tan bueno cuando se trata de variables predictoras cuantitativas. 

$$P(A|B) = \frac{P(B|A)\cdot P(A)}{P(B)}$$

**Ejemplos.**

- Reconocimiento de autoría de textos
- Detección automático de plagio
- Compra en línea de clientes (hábitos de compra)

## Función de predicción
Queremos obtener a que clase pertenece una observación basado en un vector de características para predicción (vector de contadores/frecuencias).
$$z_0 = (y_0, x_0)$$
donde $y_0$ es la clase de la observación y $x_0$ el vector predictor. La función predictora: $$f(\cdot|D)$$ donde $D$ es el conjunto de observaciones de entrenamiento. La predicción de $y_0$ es $$\hat{y_0} = f(x_0|D)$$ Recordemos que $x_0$ es un vector de contadores: cada elemento $x_0$ almacena el número de veces en que un tipo particular (categoría) o nivel de una variable cualitativa fue observada. Sea $x_{0_j}$ el número de veces que en un tipo $t_j$ aparece en el documento $F_0$. Entonces $x_0 = [x_{0,1}, \ldots, x_{0,n}]^T$ contiene las frecuencias de los tipos para el documento $F_0$ y existen $n$ tipos diferentes dentro del conjunto de tipos $T$. El propietario (clase) de $F_0$ se denota como $y_0$ y pertenece al conjnuto de propietarios: $y_0 \in {P1, \ldots, P_m}$; la propabilidad de que el tipo $t_j$ se encuentre en $F_0$ está dada por: $$\pi_{k,j} = Pr(t_j \in F_0|y_0 = P_k)$$ y esta última probabilidad es específica para cada clase pero no varía entre documentos del mismo propietario. La sumatoria de $\pi_{k,j}$ es igual a $1$. 

Finalmente, la función de predicción se define como: $$\hat{y_0} = f(x_0|D) = \text{arg máx}\{Pr(x_0|y_0 = P_1), \ldots, Pr(x_0|y_0 = P_m)\}$$

## Probabilidad posterior

La función de Bayes se puede mejorar al considerar información previa. $$Pr(y_0 = P_k | x_0) = \frac{P_r(x_0|y_0 = P_k) \times P_r(y_0 = P_k)}{P_r(x_0)}$$ y finalmente $$\hat{y_0} = \text{arg máx}\{P_r(x_0|y_0 = P_k) \times \ldots\} = \ldots = \text{arg máx}\left\{\log (\hat{\pi}_k) + \sum_{j=1}^nx_{0,j}\log (\hat{\pi}_{k,j})\right\}$$

## Detector de spam

URL dataset: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

Utilizaremos *Bag of Words* (BoW) que es un término usado para ...

### Ejemplo sencillo de CountVectorizer

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [7]:
documents = [   'Hello, how are you', 
                'Win money, win from home', 
                'Call me now', 
                'Hello, can I call you?'
            ]
from sklearn.feature_extraction.text import CountVectorizer
count_ = CountVectorizer()
print(count_)

CountVectorizer()


In [8]:
count_.fit(documents) #aplica el conteo de las observaciones
names = count_.get_feature_names()
names

['are',
 'call',
 'can',
 'from',
 'hello',
 'home',
 'how',
 'me',
 'money',
 'now',
 'win',
 'you']

In [9]:
doc_array = count_.transform(documents).toarray()
doc_array

array([[1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 2, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1]])

In [10]:
frequency_matrix = pd.DataFrame(data=doc_array, columns=names) #nos va a indicar la frecuencia de cada palabra en cada documento
frequency_matrix #los renglones son el número de documento

,are,call,can,from,hello,home,how,me,money,now,win,you
0,1,0,0,0,1,0,1,0,0,0,0,1
1,0,0,0,1,0,1,0,0,1,0,2,0
2,0,1,0,0,0,0,0,1,0,1,0,0
3,0,1,1,0,1,0,0,0,0,0,0,1


### Inicio del ejemplo con el dataset de SMS

In [11]:
df = pd.read_csv('https://bit.ly/2kCy3CN', 
                   sep='\t', 
                   names=['label','sms_message'])
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [12]:
df.shape

(5572, 2)

### Preprocesamiento

Etiquetas de texto -> Etiquetas numéricas

In [13]:
df.label = df.label.map({'ham':0, 'spam':1})
df.head()

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


### Conjuntos de entrenamiento y prueba

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.sms_message, df.label, random_state=1)
print(f'Completo      : {df.shape[0]}')
print(f'Entrenamiento : {X_train.shape[0]}')
print(f'Pruebas       : {X_test.shape[0]}')

Completo      : 5572
Entrenamiento : 4179
Pruebas       : 1393


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
training_ = count_vector.fit_transform(X_train)
testing_ = count_vector.transform(X_test)

### Predictor Bayesiano

In [24]:
from sklearn.naive_bayes import MultinomialNB
naive_ = MultinomialNB()
naive_.fit(training_, y_train)

MultinomialNB()

In [25]:
predictions_ = naive_.predict(testing_)
predictions_

array([0, 0, 0, ..., 0, 1, 0])

### Evaluación del modelo

In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy  :', format(accuracy_score(y_test, predictions_)))
print(f'Precision : {precision_score(y_test, predictions_)}')
print('Recall    :', format(recall_score(y_test, predictions_)))
print(f'F1        : {f1_score(y_test, predictions_)}')

Accuracy  : 0.9885139985642498
Precision : 0.9720670391061452
Recall    : 0.9405405405405406
F1        : 0.9560439560439562
